<a href="https://colab.research.google.com/github/albhoe/701_assignments/blob/main/project/911Call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import numpy as np
import pandas as pd
import datetime
import requests
from pathlib import Path
import json

In [101]:
global censusblocks
censusblocks = pd.DataFrame()
censusblocks.empty

def UpdateCensusBlocks():
  response = requests.get("https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/US_Census_Blocks_v1/FeatureServer/0/query?where=1%3D1&outFields=STATE,COUNTY,TRACT,BLOCK,GEOID,NAME,CENTLON,CENTLAT,INTPTLON,INTPTLAT&returnGeometry=false&outSR=4326&f=json")


  data = {index:entry['attributes'] for index,entry in enumerate(response.json()['features'])}
  df = pd.DataFrame.from_dict(data,orient='index')

  data_path = Path('./data')
  if not data_path.exists():
      data_path.mkdir()
  with open('./data/blockgroups.csv','w') as f:
    f.write(df.to_csv())

  return

def CensusBlocktoCoord(censusblock):
  global censusblocks
  if censusblocks.empty:
    with open('./data/blockgroups.csv', 'r') as f:
      censusblocks = pd.read_csv(f)
  lat_long = censusblocks.loc[censusblock == censusblocks.GEOID][['CENTLON','CENTLAT']]
  return lat_long.iloc[0,0],lat_long.iloc[0,1]

CensusBlocktoCoord(171239612002042)

(-89.3641617, 41.1143081)

In [102]:
def UpdateAnalyzeBoston():
  #"""Downloads the latest 911 dispatch data and updates the DataFrame."""
  url = "https://data.boston.gov/dataset/17129fad-fff9-4eac-ad74-51fb4bf63c22/resource/2459542e-7026-48e2-9128-ca29dd3bebf8/download/911-daily-dispatch-count-by-agency.csv"
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Assuming the data is a CSV, load it into a pandas DataFrame
    data_path = Path('./data')
    if not data_path.exists():
      data_path.mkdir()
    with open("/content/data/911-daily-dispatch-count-by-agency.csv", "wb") as f:
      f.write(response.content)

  except requests.exceptions.RequestException as e:
    print(f"Error downloading data: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")

In [103]:
def UpdateAustinTexas():
  #"""Downloads the latest 911 dispatch data and updates the DataFrame."""
  url = "https://data.austintexas.gov/resource/fsje-8gq2.csv"
  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Assuming the data is a CSV, load it into a pandas DataFrame
    data_path = Path('./data')
    if not data_path.exists():
      data_path.mkdir()
    with open("/content/data/APDAverageResponseTimebyDayandHour.csv", "wb") as f:
      f.write(response.content)

  except requests.exceptions.RequestException as e:
    print(f"Error downloading data: {e}")
  except Exception as e:
    print(f"An error occurred: {e}")
UpdateAustinTexas()

In [104]:
def CleanAustinTexas():
  df = pd.read_csv("/content/data/APDAverageResponseTimebyDayandHour.csv")
  display(df)
  return 0
CleanAustinTexas()

,incident_number,council_district,incident_type,mental_health_flag,priority_level,response_datetime,response_day_of_week,response_hour,first_unit_arrived_datetime,call_closed_datetime,...,number_of_units_arrived,unit_time_on_scene,call_disposition_description,report_written_flag,response_time,officer_injured_killed_count,subject_injured_killed_count,other_injured_killed_count,geo_id,census_block_group
0,210390914,3,Dispatched Incident,Mental Health Incident,Priority 3,2021-02-08T15:39:41.000,Mon,15,2021-02-08T15:49:23.000,2021-02-08T17:27:33.000,...,2,7948,No Report,False,646.0,0,0,0,4.845300e+11,4.530020e+09
1,213190528,6,Dispatched Incident,Not Mental Health Incident,Priority 2,2021-11-15T10:35:44.000,Mon,10,2021-11-15T11:27:08.000,2021-11-15T11:34:16.000,...,1,428,No Report,False,3109.0,0,0,0,4.849102e+11,4.910203e+09
2,190361790,4,Dispatched Incident,Not Mental Health Incident,Priority 2,2019-02-05T21:27:53.000,Tue,21,2019-02-05T21:45:54.000,2019-02-05T21:47:47.000,...,1,113,No Report,False,1091.0,0,0,0,4.845304e+11,4.530400e+09
3,140571346,Other,Dispatched Incident,Not Mental Health Incident,Priority 0,2014-02-26T20:29:31.000,Wed,20,2014-02-26T20:34:20.000,2014-02-26T20:56:35.000,...,3,2718,Unable To Locate,False,349.0,0,0,0,4.845303e+11,4.530317e+09
4,141450671,Other,Dispatched Incident,Not Mental Health Incident,Priority 3,2014-05-25T09:25:29.000,Sun,9,2014-05-25T09:33:32.000,2014-05-25T09:37:42.000,...,1,250,No Report,False,489.0,0,0,0,4.845304e+11,4.530434e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,153531579,5,Officer-Initiated Incident,Not Mental Health Incident,Priority 3,2015-12-19T21:35:19.000,Sat,21,2015-12-19T21:35:19.000,2015-12-19T21:48:27.000,...,1,788,10/8 From Traffic,False,NaN,0,0,0,4.845300e+11,4.530020e+09
996,153640600,Other,Officer-Initiated Incident,Not Mental Health Incident,Priority 3,2015-12-30T11:04:44.000,Wed,11,2015-12-30T11:04:45.000,2015-12-30T11:37:53.000,...,1,1988,No Report,False,NaN,0,0,0,NaN,NaN
997,160041710,2,Dispatched Incident,Not Mental Health Incident,Priority 3,2016-01-04T21:42:47.000,Mon,21,2016-01-04T21:45:43.000,2016-01-04T21:46:36.000,...,1,53,No Report,False,176.0,0,0,0,4.845398e+11,4.539800e+09
998,153300446,2,Dispatched Incident,Not Mental Health Incident,Priority 3,2015-11-26T09:36:13.000,Thu,9,2015-11-26T10:20:01.000,2015-11-26T10:23:32.000,...,1,211,No Report,False,2649.0,0,0,0,4.845300e+11,4.530025e+09


0

In [107]:
def CleanAnalyzeBoston():
  df = pd.read_csv("/content/data/911-daily-dispatch-count-by-agency.csv")
  display(df)
CleanAnalyzeBoston()

,Date,Year,Month,DayOfYear,Total,BPD,BFD,EMS
0,11/01/2010,2010,11,305,2603,1969,259,375
1,11/02/2010,2010,11,306,2414,1852,206,356
2,11/03/2010,2010,11,307,2487,1872,235,380
3,11/04/2010,2010,11,308,2328,1695,241,392
4,11/05/2010,2010,11,309,2478,1915,225,338
...,...,...,...,...,...,...,...,...
1263,04/17/2014,2014,4,107,3051,2383,245,423
1264,04/18/2014,2014,4,108,2951,2378,189,384
1265,04/19/2014,2014,4,109,2875,2267,205,403
1266,04/20/2014,2014,4,110,2598,1989,197,412


In [106]:
t_prev_update = datetime.datetime(1,1,1)
t_next_update = datetime.datetime(t_prev_update.year,(t_prev_update.month+1)%12,t_prev_update.day)

if datetime.datetime.now() > t_next_update:
  UpdateCensusBlocks()
  UpdateAnalyzeBoston()
  UpdateAustinTexas()
  t_prev_update = datetime.datetime.now()
  t_next_update = datetime.datetime(t_prev_update.year,(t_prev_update.month+1)%12,t_prev_update.day)